# Trabalho Final

## Código da Rede Final:

**NOTA:** Devido a versão instalada do pandas, o código gera um aviso para cada rede iterada. Deixamos esse aviso, apenas por respresentação do código rodando, mas você pode retirar este aviso se remover o comentário do código presente na última linha da célula seguinte. Entretanto, cuidado! Este código irá remover TODOS os avisos possíveis.


In [5]:
import warnings
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from itertools import product

# Carregar os dados do CSV
data = pd.read_csv('concrete.csv')

# Suprimir avisos indesejados
#warnings.filterwarnings('ignore')

**Hiperparâmetros:**

In [6]:
# Hiperparâmetros editáveis
DIC_ATIVACOES = {
    'ReLU' : nn.ReLU(),
    'Tanh' : nn.Tanh(),
    'Sigmoid': nn.Sigmoid(),
    'Softmax': nn.Softmax(),
    'GELU': nn.GELU()
}

random_state = 100                    # Definindo random_state como 100
lrs = [0.01, 0.02, 0.05, 0.1, 0.15]   # Definindo diferentes taxas de aprendizado
epochs = [100, 150, 200]              # Adicionando mais opções para número de épocas
hidden_sizes = [16, 32, 64, 128, 256] # Adicionando diferentes números de neurônios na camada oculta
dropout_prob = 0.5                    # Probabilidade de dropout
k_folds = 5                           # Número de folds para validação cruzada

**Rede geradora de `.csv`:**

In [11]:
# Separar atributos (X) e target (y)
X = data.drop(columns=["cement", "slag", "ash", "water", "superplastic", "coarseagg", "fineagg", "age"])
y = data['strength']

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Converter para tensores PyTorch
X = torch.tensor(X_scaled, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.float32)

# Dataframe para armazenar os resultados
results_df = pd.DataFrame(columns=['Folds number',
                                   'Fold', 
                                   'Random State',
                                   'Dropout probability', 
                                   'Activation function', 
                                   'Learning Rate', 
                                   'Epochs', 
                                   'Layers number', 
                                   'Hidden Size', 
                                   'RMSE'
                                  ])

# K-Fold Cross-Validation
kf = KFold(n_splits=k_folds, shuffle=True, random_state=random_state)

# Criar todas as combinações possíveis dos hiperparâmetros
hyperparameters_combinations = product(lrs, epochs, hidden_sizes, range(1, 5), DIC_ATIVACOES.items())

# Loop pelos hiperparâmetros
for fold, (train_index, test_index) in enumerate(kf.split(X)):
    # Dividir os dados em conjuntos de treinamento e teste para o fold atual
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    for lr, num_epochs, hidden_size, num_layers, (ativacao_name, ATIVACAO) in hyperparameters_combinations:
        # Definir a classe da rede MLP
        class MLP(nn.Module):
            def __init__(self, input_size, hidden_size, output_size, num_layers, dropout_prob=dropout_prob): 
                super(MLP, self).__init__()
                self.layers = nn.ModuleList()
                self.layers.append(nn.Linear(input_size, hidden_size))

                for _ in range(num_layers):
                    self.layers.append(nn.Linear(hidden_size, hidden_size))
                    self.layers.append(nn.Dropout(dropout_prob))  # Adicionando Dropout

                self.layers.append(nn.Linear(hidden_size, output_size))
                self.activation = ATIVACAO

            def forward(self, x):
                for layer in self.layers[:-1]:
                    x = self.activation(layer(x))
                x = self.layers[-1](x)
                return x

        # Parâmetros da rede
        input_size = X_train.shape[1]
        output_size = 1  # Problema de regressão

        # Criar a rede
        model = MLP(input_size, hidden_size, output_size, num_layers, dropout_prob)

        # Definir a função de custo e o otimizador
        criterion = nn.MSELoss()  # Mudando para MSELoss já que é uma tarefa de regressão
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Treinamento da rede
        for epoch in range(num_epochs):
            model.train()
            optimizer.zero_grad()
            outputs = model(X_train)
            loss = criterion(outputs.squeeze(), y_train)
            loss.backward()
            optimizer.step()

        # Avaliação do modelo
        model.eval()
        with torch.no_grad():
            outputs = model(X_test).squeeze()
            mse = mean_squared_error(y_test, outputs)
            RMSE = np.sqrt(mse)

        # Armazenar os resultados
        results_df = results_df.append({'Folds number': k_folds,
                                        'Fold': fold + 1,
                                        'Random State': random_state,
                                        'Dropout probability': dropout_prob,
                                        'Activation function': ativacao_name, 
                                        'Learning Rate': lr,
                                        'Epochs': num_epochs, 
                                        'Layers number': num_layers,
                                        'Hidden Size': hidden_size,
                                        'RMSE': RMSE,
                                    }, ignore_index=True)

# Salvar os resultados em um arquivo CSV
results_df.to_csv('results3.csv', index=False)

/tmp/ipykernel_3095452/2750539514.py:119: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Folds number': k_folds,
/tmp/ipykernel_3095452/2750539514.py:119: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Folds number': k_folds,
/tmp/ipykernel_3095452/2750539514.py:119: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({'Folds number': k_folds,
/home/sergio23040/anaconda3/envs/ilumpy/lib/python3.11/site-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/tmp/ipykernel_3095452/2750539514

## Análise dos hiperparâmetros:

In [1]:
import pandas as pd

In [2]:
results_df = pd.read_csv('results3.csv')

# Encontrar a linha com o melhor RMSE
best_params = results_df.loc[results_df['RMSE'].idxmin()]
print("\nMelhores parâmetros de RMSE:")
print(best_params)

# Encontrar a linha com o pior RMSE
worst_params = results_df.loc[results_df['RMSE'].idxmax()]
print("\nPiores parâmetros de RMSE:")
print(worst_params)

# Calcular a média dos RMSEs
mean_rmse = results_df['RMSE'].mean()
print("\nMédia total de RMSE:", mean_rmse)

# Calcular a média do RMSE para cada função de ativação
mean_rmse_activation = results_df.groupby('Activation function')['RMSE'].mean()
print("\nMédia do RMSE para cada função de ativação:")
print(mean_rmse_activation)

# Calcular a média do RMSE para cada número de camadas
mean_rmse_layers = results_df.groupby('Layers number')['RMSE'].mean()
print("\nMédia do RMSE para cada número de camadas:")
print(mean_rmse_layers)

# Calcular a média do RMSE para cada valor de aprendizagem
mean_rmse_lr = results_df.groupby('Learning Rate')['RMSE'].mean()
print("\nMédia de RMSE para cada valor de aprendizagem:")
print(mean_rmse_lr)

# Calcular a média do RMSE para cada número de épocas
mean_rmse_epochs = results_df.groupby('Epochs')['RMSE'].mean()
print("\nMédia de RMSE para cada número de épocas:")
print(mean_rmse_epochs)

# Calcular a média do RMSE para cada tamanho de camada interna
mean_rmse_hidden_size = results_df.groupby('Hidden Size')['RMSE'].mean()
print("\nMédia do RMSE para cada tamanho de camada interna:")
print(mean_rmse_hidden_size)


Melhores parâmetros de RMSE:
Folds number                  5
Fold                          1
Random State                100
Dropout probability         0.5
Activation function        ReLU
Learning Rate              0.05
Epochs                      200
Layers number                 1
Hidden Size                 256
RMSE                   0.106414
Name: 880, dtype: object

Piores parâmetros de RMSE:
Folds number                  5
Fold                          1
Random State                100
Dropout probability         0.5
Activation function        ReLU
Learning Rate              0.15
Epochs                      100
Layers number                 4
Hidden Size                 256
RMSE                   40.50468
Name: 1295, dtype: object

Média total de RMSE: 16.93279981701

Média do RMSE para cada função de ativação:
Activation function
GELU        6.895653
ReLU       11.490613
Sigmoid    16.953957
Softmax    27.937414
Tanh       21.386362
Name: RMSE, dtype: float64

Média do RMSE pa

## Referências

- presentes no github deste documento
